In [1]:
from bs4 import BeautifulSoup as bs
import datetime as dt
import pandas as pd
import requests
import warnings

warnings.filterwarnings('ignore')

In [2]:
cookies = {
    '__ddg1_': 'KWn3c1WwZzj6FysUsfDd',
    'ASPNET_SessionID': '5gqp1nhoznc4coucajtcdmt5',
    '_ym_uid': '172173650670206345',
    '_ym_d': '1721736506',
    '_ym_isad': '1',
    '_ym_visorc': 'b',
    'accept': '1',
}

headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'cache-control': 'max-age=0',
    # 'cookie': '__ddg1_=KWn3c1WwZzj6FysUsfDd; ASPNET_SessionID=5gqp1nhoznc4coucajtcdmt5; _ym_uid=172173650670206345; _ym_d=1721736506; _ym_isad=1; _ym_visorc=b; accept=1',
    'priority': 'u=0, i',
    'referer': 'https://www.cbr.ru/currency_base/',
    'sec-ch-ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
}

In [3]:
link = 'https://www.cbr.ru/hd_base/metall/metall_base_new/?UniDbQuery.Posted=True&UniDbQuery.From=01.01.2014&UniDbQuery.To=23.07.2024&UniDbQuery.Gold=true&UniDbQuery.Silver=true&UniDbQuery.Platinum=true&UniDbQuery.Palladium=true&UniDbQuery.so=1'

In [4]:
response = requests.get(link, cookies=cookies, headers=headers).text
soup = bs(response, 'html.parser')

list_header = []
data = []

In [5]:
header = soup.find_all("table")[0].find("tr")

for items in header:
    try: list_header.append(items.get_text())
    except: continue
HTML_data = soup.find_all("table")[0].find_all("tr")[1:]

for element in HTML_data:
    sub_data = []
    for sub_element in element:
        try:
            sub_data.append(sub_element.get_text())
        except:
            continue
    data.append(sub_data)
    
df = pd.DataFrame(data=data, columns=list_header)

In [6]:
df.rename(columns={'Дата* ▼\n': 'Дата'}, inplace=True)

In [7]:
df = df[['Дата', 'Золото', 'Серебро', 'Платина', 'Палладий']]
df['Дата'] = pd.to_datetime(df['Дата'], format='%d.%m.%Y')

In [8]:
for name in df.columns[1:]:
    df[name] = df[name].str.replace(',', '.').str.replace(' ', '').astype(float)

In [9]:
df

,Дата,Золото,Серебро,Платина,Палладий
0,2024-07-23,6783.17,82.14,2723.43,2593.61
1,2024-07-20,6972.37,86.23,2818.61,2688.43
2,2024-07-19,7007.35,87.34,2881.76,2743.33
3,2024-07-18,6919.31,87.41,2781.09,2659.31
4,2024-07-17,6872.34,87.25,2815.64,2713.46
...,...,...,...,...,...
2595,2014-01-15,1334.47,21.47,1527.09,786.52
2596,2014-01-14,1326.80,21.08,1526.99,787.99
2597,2014-01-11,1315.55,20.86,1517.06,785.76
2598,2014-01-10,1306.85,20.81,1507.25,785.60
